In [0]:
import tensorflow as tf
import numpy as np

In [0]:
with open("anna.txt") as f:
  text=f.read()
  
vocab=sorted(set(text))
vocab_2_int={ch:i for i,ch in enumerate(vocab)}
int_2_vocab=dict(enumerate(vocab))
with open("ann.txt") as f:
  text1=f.read()
encoded=np.array([vocab_2_int[ch] for ch in text1],dtype=np.int32)

In [35]:
text[:100]


'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [36]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70],
      dtype=int32)

In [37]:
len(vocab)

83

In [0]:
def get_batches(arr,batch_size,n_steps):
  
  chars_per_batch=batch_size*n_steps
  n_batches=len(arr)//chars_per_batch
  
  arr=arr[:n_batches*chars_per_batch]
  # Reshape into batch_size rows
  arr=arr.reshape((batch_size,-1))
  
  for n in range(0,arr.shape[1],n_steps):
    x=arr[:,n:n+n_steps]
    
    y_temp=arr[:,n+1:n+n_steps+1]
    
    y=np.zeros(x.shape,dtype=x.dtype)
    
    y[:,:y_temp.shape[1]]=y_temp
    
    yield x,y
  
  

In [0]:
batches=get_batches(encoded,10,50)
x,y=next(batches)

In [40]:
print("x=\n",x[:10,:10])
print("\ny=\n",y[:10,:10])

x=
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y=
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


In [0]:
def build_inputs(batch_size,num_steps):
  inputs=tf.placeholder(tf.int32,[batch_size,num_steps],name="inputs")
  target=tf.placeholder(tf.int32,[batch_size,num_steps],name="targets")
  
  keep_prob=tf.placeholder(tf.float32,name="keep_prob")
  
  return inputs,target,keep_prob

In [0]:
def build_lstm(lstm_size,num_layers,batch_size,keep_prob):
  
  def build_cell(lstm_size,keep_prob):
    lstm=tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop=tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=keep_prob)
    
    return drop
  #stack multiple lstm layers
  cell=tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size,keep_prob) for _ in range(num_layers)])
  
  initial_state=cell.zero_state(batch_size,tf.float32)
  
  return cell,initial_state
  
  

In [0]:
def build_output(lstm_output,in_size,out_size):
#   in_size: Size of the input tensor, for example, size of the LSTM cells
#   out_size: Size of this softmax layer
  
  seq_output=tf.concat(lstm_output,axis=1)
#   x :input tensor
  x=tf.reshape(seq_output,[-1,in_size])
  
  with tf.variable_scope('softmax'):
    weight=tf.Variable(tf.truncated_normal((in_size,out_size),stddev=0.1))
    bias=tf.Variable(tf.zeros(out_size))
    
    
  logits=tf.matmul(x,weight)+bias
  
  out=tf.nn.softmax(logits,name="prediction")
  
  return out,logits

In [0]:
def build_loss(logits,targets,lstm_size,num_characters):
  
  y_one_hot=tf.one_hot(targets,num_characters)
  y_reshaped=tf.reshape(y_one_hot,logits.get_shape())
  
  loss=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_reshaped)
  loss=tf.reduce_mean(loss)
  
  return loss

In [0]:
def build_optimizer(loss,learning_rate,grad_clip):
  tvars=tf.trainable_variables()
  grads,_=tf.clip_by_global_norm(tf.gradients(loss,tvars),grad_clip)
  train_optimizer=tf.train.AdamOptimizer(learning_rate)
  optimizer=train_optimizer.apply_gradients(zip(grads,tvars))
  
  return optimizer

In [0]:
class charRNN:
  def __init__(self,num_characters,batch_size=64,num_steps=50,lstm_size=128,num_layers=2,learning_rate=0.01,grad_clip=5,sampling=False):
    
    if sampling==True:
      batch_size,num_steps=1,1
      
    else:
      batch_size,num_steps=batch_size,num_steps
      
    tf.reset_default_graph()
    
    self.inputs,self.targets,self.keep_prob=build_inputs(batch_size,num_steps)
    
    cell,self.initial_state=build_lstm(lstm_size,num_layers,batch_size,self.keep_prob)
    
    #     Run the data through the RNN layers
    #     First, one-hot encode the input tokens
    
    x_one_hot=tf.one_hot(self.inputs,num_characters)
    
    # Run each sequence step through the RNN and collect the outputs
    
    outputs,state=tf.nn.dynamic_rnn(cell,x_one_hot,initial_state=self.initial_state)  
    self.final_state=state
    
    self.prediction,self.logits=build_output(outputs,lstm_size,num_characters)
    
    self.loss=build_loss(self.logits,self.targets,lstm_size,num_characters)
    
    self.optimizer=build_optimizer(self.loss,learning_rate,grad_clip)
    
    
    
    
    
    

In [0]:
batch_size=100
num_steps=100
num_layers=2
lstm_size=512
keep_prob=0.5
learning_rate=0.001
epochs=10

In [48]:

model = charRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver=tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  count=0
  for e in range(epochs):
    new_state=sess.run(model.initial_state)
    loss=0
    
    for x,y in get_batches(encoded,batch_size,num_steps):
      count=count+1
      feed_dict={model.inputs:x,model.targets:y,model.keep_prob:keep_prob,model.initial_state:new_state}
      
      batch_loss,new_state,_ = sess.run([model.loss,model.final_state,model.optimizer],feed_dict=feed_dict)
      
      if(count%50==0):
        print("Epoch:{0} , loss:{1}".format(e+1,batch_loss))
        
      if(count%200==0):
        saver.save(sess,"checkpoints/i{}_l{}.ckpt".format(count,lstm_size))
        
  saver.save(sess,"checkpoints1/i{}_l{}.ckpt".format(count,lstm_size))
      
  

Epoch:1 , loss:3.1710784435272217
Epoch:1 , loss:3.095881938934326
Epoch:1 , loss:2.7897090911865234
Epoch:2 , loss:2.4521093368530273
Epoch:2 , loss:2.3592545986175537
Epoch:2 , loss:2.237067461013794
Epoch:2 , loss:2.190321922302246
Epoch:3 , loss:2.059110403060913
Epoch:3 , loss:1.9929144382476807
Epoch:3 , loss:1.9267240762710571
Epoch:3 , loss:1.907214641571045
Epoch:4 , loss:1.8006675243377686
Epoch:4 , loss:1.8037002086639404
Epoch:4 , loss:1.7658149003982544
Epoch:4 , loss:1.7339824438095093
Epoch:5 , loss:1.6924253702163696
Epoch:5 , loss:1.648697853088379
Epoch:5 , loss:1.6302270889282227
Epoch:5 , loss:1.611778736114502
Epoch:6 , loss:1.5825355052947998
Epoch:6 , loss:1.6020562648773193
Epoch:6 , loss:1.5482124090194702
Epoch:6 , loss:1.5497808456420898
Epoch:7 , loss:1.5092449188232422
Epoch:7 , loss:1.54668390750885
Epoch:7 , loss:1.4713447093963623
Epoch:7 , loss:1.4501125812530518
Epoch:8 , loss:1.472472906112671
Epoch:8 , loss:1.4405592679977417
Epoch:8 , loss:1.3889554

In [49]:
tf.train.get_checkpoint_state('checkpoints1')

model_checkpoint_path: "checkpoints1/i1980_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/../checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/../checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/../checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/../checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/../checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/../checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/../checkpoints/i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/../checkpoints/i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/../checkpoints/i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints1/i1980_l512.ckpt"

In [0]:
def pick_top_n(preds,vocab_size,top_n=5):
  p=np.squeeze(preds)
  p[np.argsort(p)[:-top_n]]=0
  p=p/np.sum(p)
  c=np.random.choice(vocab_size,1,p=p)[0]
  
  return c

In [0]:
def sample(checkpoint,n_samples,lstm_size,vocab_size,prime='the '):
  
  samples=[ch for ch in prime]
  model=charRNN(len(vocab),lstm_size=lstm_size,sampling=True)
  saver=tf.train.Saver()
  
  with tf.Session() as sess:
    saver.restore(sess,checkpoint)
    new_state=sess.run(model.initial_state)
    
    for ch in prime:
      x=np.zeros((1,1))
      x[0,0]=vocab_2_int[ch]
      feed_dict={model.inputs:x , model.keep_prob:1.0 ,model.initial_state: new_state}
      preds,new_state=sess.run([model.prediction,model.final_state],feed_dict=feed_dict)

    ch=pick_top_n(preds,len(vocab))
    samples.append(int_2_vocab[ch])
    
    for i in range(n_samples):
      x[0,0]=ch
          
      feed_dict={model.inputs:x , model.keep_prob:1.0 ,model.initial_state:new_state}
    
      preds,new_state=sess.run([model.prediction,model.final_state],feed_dict=feed_dict)
      
      ch=pick_top_n(preds,len(vocab))
      samples.append(int_2_vocab[ch])
      
  return ''.join(samples) 

In [52]:
tf.train.latest_checkpoint('checkpoints1')

'checkpoints1/i1980_l512.ckpt'

In [60]:
checkpoint = tf.train.latest_checkpoint('checkpoints1')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="aunty ji dance ")
print(samp)

aunty ji dance of
their sendency of the princess with the bith oppeass of his had
and heart to her share about the something.

"What was so yut on the discussion of me at a moming, there'voute the
distrating one who were not tendenners to say anything in she should
not have taken for you to saying while the sectaterest and so
an excordent one was at him, and we are a grander so to do with
me. That, I don't see her. There will been taken this sunsericalien to
arruse to a sparetious at him, and I was a smule of the marming to
be a childrong. They had to the bord was strects of mistaken,
and he had all all things the charemunass, but so they had been always
told them.


Consters out of which, husband all has been seeing her. And the
praces of the place as she was the solition of his soul in a stort
to see all. The carriage and something was in the same.

The share one of the propiced said, and then they were set in her
exacker. "In there's a girl when the morning is at it is attacking the